In [1]:
import sys
sys.path.insert(0, '/home/mnetlab/anaconda3/envs/pp_fl/lib/python3.6/site-packages')
sys.path

['/home/mnetlab/anaconda3/envs/pp_fl/lib/python3.6/site-packages',
 '/home/mnetlab/anaconda3/envs/pp_fl/lib/python36.zip',
 '/home/mnetlab/anaconda3/envs/pp_fl/lib/python3.6',
 '/home/mnetlab/anaconda3/envs/pp_fl/lib/python3.6/lib-dynload',
 '',
 '/home/mnetlab/.local/lib/python3.6/site-packages',
 '/home/mnetlab/anaconda3/envs/pp_fl/lib/python3.6/site-packages',
 '/home/mnetlab/.local/lib/python3.6/site-packages/IPython/extensions',
 '/home/mnetlab/.ipython']

In [2]:
# Modified fedavg for our input
from __future__ import absolute_import, division, print_function, unicode_literals

# Other import
import sys
import os
import tensorflow as tf
from data.preprocessing import preprocessing_for_training, separate_and_preprocess_for_simple_fed, evaluate_with_new_model
from data.read_data import read_data, read_setting
from data.data_utils import load_cifar10_data, train_test_label_to_categorical
from model.model import init_model, record_history, training_once, print_result_for_fed
from model.operation import broadcast_to_device, caculate_delta, aggregate_add, aggregate_division_return 
from config.environment import gpu_decision, zmq_bind

from math import floor
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator

import multiprocessing as mp
import zmq

Using TensorFlow backend.


In [3]:
def step_decay(epoch):
    epoch = _ + epo
    # initial_lrate = 1.0 # no longer needed
    drop = 0.99
    epochs_drop = 1.0
    
    lrate = 0.2 * pow(drop, floor((1+epoch)/epochs_drop))
    return lrate

In [4]:
_, epo = 0, 0

In [5]:
def model_training(r, device):
    # Load CIFAR-10 data
    train_images, train_labels, test_images, test_labels = load_cifar10_data()
    
    gpu_decision(training_info["num_gpu"] ,device)
            
    model_m = tf.keras.models.load_model('parallel/model_m.h5')

    # Define our model
    if(r == 0):
        locals()['model_{}'.format(device)] = init_model()
    else:
        locals()['model_{}'.format(device)] = tf.keras.models.load_model('parallel/model_m.h5')
    
    # Local training on each device
    for epo in range(training_info["num_local_epoch"]):
        train_new_image, train_new_label = separate_and_preprocess_for_simple_fed(device, train_images, train_labels, training_info["num_device"])
        history_temp = training_once(locals()['model_{}'.format(device)], train_new_image, train_new_label, training_info, augment, callback)

    # Calculate delta weight on each device
    caculate_delta(locals()['model_{}'.format(device)], model_m)
        
    # Save delta weight in each model weight(.h5) file 
    try:
        if(r == 0):
            locals()['model_{}'.format(device)].save('parallel/model_{}.h5'.format(device))
        else:
            locals()['model_{}'.format(device)].save_weights('parallel/model_{}.h5'.format(device))
    except:
        print("error found")

In [ ]:
# Adjust parameters of the model
callback = tf.keras.callbacks.LearningRateScheduler(step_decay)
# Define the method for preprocessing
augment = ImageDataGenerator(preprocessing_function=preprocessing_for_training)

# bind zmq socket port
socket = zmq_bind()

 # Read detailed settings from json file
detailed_setting = read_setting()
training_info = detailed_setting["training_info"]

while True:
    while True:
        # Recieve message from server
        message = socket.recv()
        ms = str(message, encoding = "utf-8").split(":")
        r = int(ms[0])
        device = int(ms[1])
        print("(Received) round: %s, device: %s" % (message, device))
        
        # Start process for the device
        locals()['p_{}'.format(device)] = mp.Process(target=model_training, args=(r, device))
        locals()['p_{}'.format(device)].start()
        
        # Send message back to master
        # Tell master that the process (device) starts local training
        socket.send(b"ok")
        if(device == (training_info['num_device']-1)): break
    
    # Wait all processes done the jobs (local training)
    for device in range(training_info['num_device']):
        locals()['p_{}'.format(device)].join()

    # Tell server that the 'r' round finishs
    message = socket.recv()
    socket.send(b"finish")
    if(r == (training_info['num_round']-1)): break

(Received) round: b'0:0', device: 0
(Received) round: b'0:1', device: 1
(Received) round: b'0:2', device: 2
(Received) round: b'0:3', device: 3








Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructorWARNING:tensorflow:From /home/mnetlab/anaconda3/envs/pp_fl/l

250/250 [==============================] - ETA: 0s - loss: 2.1736 - acc: 0.220 - 3s 12ms/step - loss: 2.1742 - acc: 0.2207


250/250 [==============================] - 2s 6ms/step - loss: 1.9347 - acc: 0.2921


250/250 [==============================] - ETA: 2s - loss: 2.0821 - acc: 0.280 - ETA: 1s - loss: 1.8664 - acc: 0.300 - ETA: 1s - loss: 1.8769 - acc: 0.316 - ETA: 1s - loss: 1.8805 - acc: 0.318 - ETA: 1s - loss: 1.8437 - acc: 0.3262 - ETA: 2s - loss: 1.8511 - acc: 0.3200 - ETA: 2s - loss: 1.8658 - acc: 0.320 37/250 [===>..........................] - ETA: 1s - loss: 1.8456 - acc: 0.3281  1/250 [..............................] - ETA: 1s - loss: 1.7254 - acc: 0.340 - ETA: 1s - loss: 1.8126 - acc: 0.346 - ETA: 1s - loss: 1.8220 - acc: 0.317  9/250 [>.............................] - ETA: 1s - loss: 1.9051 - acc: 0.3022 - ETA: 1s - loss: 1.8340 - acc: 0.337 - ETA: 1s - loss: 1.8546 - acc: 0.325 - ETA: 1s - loss: 1.8288 - acc: 0.33 - ETA: 1s - loss: 1.8639 - acc: 0.3082 - ETA: 1s - loss: 1.8330 - acc: 0.337 - ETA: 1s - loss: 1.8208 - acc: 0.327 - ETA: 1s - loss: 1.8206 - acc: 0.33 25/250 [==>...........................] 62/250 [======>.......................] - ETA: 1s - loss: 1.8379 - acc: 0.

/home/mnetlab/Jeff/model/operation.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  model_m.layers[layer].get_weights()))


...

250/250 [==============================] - 2s 7ms/step - loss: 1.0391 - acc: 0.6606


217/250 [=========================>....] - ETA: 0s - loss: 1.0450 - acc: 0.6523 - ETA: 0s - loss: 1.0441 - acc: 0.650 - ETA: 0s - loss: 1.0327 - acc: 0.6540

/home/mnetlab/Jeff/model/operation.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  model_m.layers[layer].get_weights()))


244/250 [============================>.] - ETA: 0s - loss: 1.0312 - acc: 0.65250/250 [==============================]250/250 [==============================] - 2s 6ms/step - loss: 1.0319 - acc: 0.6568
 - 2s 6ms/step - loss: 1.0423 - acc: 0.6550


/home/mnetlab/Jeff/model/operation.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  model_m.layers[layer].get_weights()))
/home/mnetlab/Jeff/model/operation.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  model_m.layers[layer].get_weights()))
/home/mnetlab/Jeff/model/operation.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray